# Choropleth Maps and Geographical Data Visualization with GeoPandas

This tutorial covers essential techniques for visualizing geographic data in Python, from basic maps to sophisticated choropleth visualizations. By the end of this tutorial, you'll be able to create informative, visually appealing maps that effectively communicate spatial patterns in your data.

## Learning Objectives
- Understand fundamental concepts of geospatial data visualization
- Load and manipulate different types of spatial data (points, lines, polygons)
- Create basic maps and control their visual appearance
- Develop choropleth maps using various classification methods
- Apply advanced techniques like map insets and focused views
- Learn best practices for effective geographical data visualization

In [ ]:
# Import necessary libraries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import geopandas as gpd
import seaborn as sns
from mapclassify import EqualInterval, Quantiles, FisherJenks
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

# Set matplotlib to display high-quality figures
%config InlineBackend.figure_format = 'retina'
plt.rcParams['figure.dpi'] = 100
plt.rcParams['savefig.dpi'] = 200

# Define a consistent color palette for the tutorial
COLORS = {
    'background': '#e6e6e6',
    'borders': '#666666',
    'highlight': '#d60000',
    'points': '#003366',
    'water': '#a6cee3'
}

# Define standard colormaps for consistent visualization
SEQUENTIAL_CMAP = 'YlOrRd'  # For continuous data like crime rates
CATEGORICAL_CMAP = 'Set3'   # For categorical data

In [ ]:
# Check versions of libraries
import matplotlib
print(f"Matplotlib version: {matplotlib.__version__}")
print(f"GeoPandas version: {gpd.__version__}")

## 1. Introduction to Spatial Data Visualization

**Geographic Information Systems (GIS)** help us analyze and visualize location-based data. GeoPandas extends the capabilities of pandas to handle spatial data, making it easier to work with geographic information in Python.

### What is Spatial Data?

Spatial data contains information about the location and shape of objects in space. It generally falls into three main categories:

1. **Vector Data**: Represents discrete features using coordinates
   - **Points**: Single locations (e.g., buildings, cities)
   - **Lines**: Connected points (e.g., roads, rivers)
   - **Polygons**: Enclosed areas (e.g., administrative boundaries, lakes)

2. **Raster Data**: Grid of cells with values (e.g., satellite imagery, elevation models)

3. **Attribute Data**: Non-spatial information associated with spatial features

### Key Capabilities of GeoPandas

- Loading spatial data from various formats (Shapefiles, GeoJSON, etc.)
- Performing spatial operations (unions, intersections, buffers)
- Converting between different coordinate reference systems (CRS)
- Creating informative and visually appealing maps
- Analyzing spatial relationships between features

## 2. Loading and Exploring Spatial Data

The foundation of any geospatial analysis is the data. GeoPandas uses the `GeoDataFrame` object, which extends pandas' `DataFrame` with geometry capabilities. Each row in a GeoDataFrame represents a feature, with one special column called `geometry` that contains the spatial information.

### Common File Formats in GIS

- **Shapefile (.shp)**: Traditional format developed by Esri
- **GeoJSON (.geojson)**: Lightweight format based on JSON
- **GeoPackage (.gpkg)**: Open, non-proprietary format
- **KML (.kml)**: Used by Google Earth and other applications

Let's start exploring different geometric types (polygons, lines, and points) by loading and visualizing them.

In [ ]:
# Load London LSOA boundaries (polygons)
# LSOAs (Lower Super Output Areas) are small geographic areas in the UK
# designed for consistent statistical analysis with ~1,500 residents each
lsoas_london_link = 'data/shp/LSOA_2011_London_gen_MHW.shp'
lsoas_london = gpd.read_file(lsoas_london_link)

In [ ]:
# Examine the data structure
print(f"Number of LSOAs: {len(lsoas_london)}")
print(f"Coordinate Reference System: {lsoas_london.crs}")
print("\nData sample:")
lsoas_london.head()

In [ ]:
# Basic plot of LSOA polygons
fig, ax = plt.subplots(figsize=(10, 8))
lsoas_london.plot(ax=ax)
plt.title('London LSOAs (Basic Plot)')
plt.axis('off')
plt.tight_layout()
plt.show()

This simple plot provides a quick view of the data structure, showing the boundaries of all LSOAs in London. While not aesthetically refined, it helps us verify that our data has loaded correctly and has the expected spatial coverage.

The plot appears as a detailed outline of Greater London, divided into many small polygons. Each polygon represents an LSOA, which is a census geography unit used for statistical analysis in the UK.

### Understanding Geometries

Each row in a GeoDataFrame has a geometry that defines its shape and location. Let's examine a single geometry to understand its structure:

In [ ]:
# Plot a single LSOA polygon
single_lsoa = lsoas_london.loc[0]
print(f"LSOA code: {single_lsoa['LSOA11CD']}")
print(f"LSOA name: {single_lsoa['LSOA11NM']}")
print(f"Geometry type: {single_lsoa['geometry'].geom_type}")
print(f"Area: {single_lsoa['geometry'].area:.2f} square units")

# Display the geometry
single_lsoa['geometry']

### 2.1 Working with Different Geometry Types

Geographic data comes in various forms. Let's explore line and point data to complement our polygon dataset.

**Lines Example: London Underground Network**

Line geometries represent linear features like roads, rivers, or in this case, the London Underground (tube) network.

In [ ]:
# Load London tube lines (lines)
london_tube_link = 'data/shp/london_tublines.shp'
london_tube = gpd.read_file(london_tube_link)

In [ ]:
# Examine tube line data
print(f"Number of tube lines: {len(london_tube)}")
print(f"Available attributes: {london_tube.columns.tolist()}")
london_tube.head()

In [ ]:
# Basic plot of tube lines with improved styling
fig, ax = plt.subplots(figsize=(10, 8))

# Plot the tube lines with a distinctive color
london_tube.plot(
    ax=ax, 
    color=COLORS['highlight'],
    linewidth=1.5,
    alpha=0.8
)

plt.title('London Underground Network', fontsize=14)
plt.axis('off')
plt.tight_layout()
plt.show()

**Points Example: Points of Interest**

Point geometries represent discrete locations such as cities, buildings, or points of interest. Let's examine some points of interest in London.

In [ ]:
# Load points of interest (points)
london_pois_link = 'data/shp/gis.osm_pois_free_1.shp'
london_pois = gpd.read_file(london_pois_link)

In [ ]:
# Examine POI data
print(f"Number of POIs: {len(london_pois)}")
print(f"Original CRS: {london_pois.crs}")
london_pois.head()

In [ ]:
# Convert to British National Grid coordinate system (EPSG:27700)
# Ensuring consistent projections is crucial when combining different datasets
london_pois = london_pois.to_crs('epsg:27700')
print(f"New CRS: {london_pois.crs}")

In [ ]:
# Verify the coordinate transformation 
# Note how coordinates are now in meters rather than decimal degrees
london_pois.head()

In [ ]:
# Check unique categories of points
poi_categories = london_pois['fclass'].value_counts()
print(f"Number of POI categories: {len(poi_categories)}")
print("\nTop 10 categories:")
print(poi_categories.head(10))

In [ ]:
# Filter to just show pubs - a quintessential British POI!
london_pubs = london_pois.loc[london_pois['fclass']=="pub"]
print(f"Number of pubs in dataset: {len(london_pubs)}")
london_pubs.info()

In [ ]:
# Plot pubs as points with improved styling
fig, ax = plt.subplots(figsize=(10, 8))

# Create a more visually appealing representation of pub locations
london_pubs.plot(
    ax=ax, 
    color=COLORS['points'],
    markersize=3,
    alpha=0.7,
    edgecolor='black',
    linewidth=0.1
)

plt.title('London Pubs', fontsize=14)
plt.axis('off')
plt.tight_layout()
plt.show()

## 3. Basic Map Styling

Creating effective maps involves much more than simply plotting data. Careful attention to visual elements can significantly enhance readability and impact. Here are some key aspects to consider:

### Elements of Map Design

- **Color**: Choose appropriate color schemes for your data type
  - Sequential schemes for continuous data (e.g., temperature)
  - Diverging schemes for data with a meaningful midpoint (e.g., profit/loss)
  - Qualitative schemes for categorical data (e.g., land use types)

- **Layout**: Control map dimensions, legends, titles, and annotations

- **Symbology**: Use appropriate markers, line styles, and polygon fills

- **Context**: Add basemaps, scale bars, and north arrows when appropriate

Let's apply these principles to create more polished maps:

In [ ]:
# Create a more styled map of LSOAs
fig, ax = plt.subplots(1, figsize=(12, 8), subplot_kw=dict(aspect='equal'))

# Remove axis frames for a cleaner look
ax.axis('off')

# Plot with enhanced styling
lsoas_london.plot(
    ax=ax, 
    linewidth=0.1, 
    facecolor=COLORS['background'],
    edgecolor=COLORS['borders'],
    alpha=0.9
)

# Add title with improved formatting
plt.title('Greater London LSOAs (Lower Super Output Areas)', fontsize=14, fontweight='bold')

# Add a text annotation explaining what LSOAs are
ax.annotate(
    "LSOAs are small areas designed for statistical analysis with populations of ~1,500 people",
    xy=(0.5, 0.02),
    xycoords='figure fraction',
    ha='center',
    fontsize=9,
    bbox=dict(boxstyle="round,pad=0.3", fc="white", ec="gray", alpha=0.8)
)

plt.tight_layout()
plt.show()

## 4. Composing Multi-Layer Maps

One of the most powerful aspects of GIS is the ability to combine different spatial datasets into a single visualization. This layered approach allows us to explore relationships between different types of features.

### Principles of Multi-Layer Mapping

- **Layer Order**: Place larger polygons at the bottom, with lines and points on top
- **Visual Hierarchy**: Use color, transparency, and size to emphasize important features
- **Simplification**: Remove unnecessary detail from background layers
- **Consistency**: Maintain a unified visual style across layers

Let's create a multi-layer map combining our LSOA boundaries, tube lines, and pub locations:

In [ ]:
# Create a multi-layer map with LSOAs, tube lines, and pubs
fig, ax = plt.subplots(1, figsize=(12, 8), subplot_kw=dict(aspect='equal'))

# Remove axis frames
ax.axis('off')

# First layer: LSOA polygons (base layer)
lsoas_london.plot(
    ax=ax, 
    linewidth=0.05, 
    facecolor=COLORS['background'],
    edgecolor=COLORS['borders'], 
    alpha=0.8
)

# Second layer: Tube lines
london_tube.plot(
    ax=ax, 
    linewidth=0.8,
    color=COLORS['highlight'],
    alpha=0.7, 
    label='Underground Lines'
)

# Third layer: Pubs (a sample of them to avoid overplotting)
# We'll sample 1000 pubs for better visualization
pub_sample = london_pubs.sample(min(1000, len(london_pubs)), random_state=42)
pub_sample.plot(
    ax=ax,
    markersize=3,
    color=COLORS['points'],
    alpha=0.7, 
    label='Pubs'
)

# Add legend with improved styling
legend = ax.legend(
    loc='lower right', 
    frameon=True, 
    framealpha=0.9,
    title="Map Features"
)
legend.get_title().set_fontweight('bold')

# Add title and annotation
plt.title('London: Administrative Boundaries, Transport Network, and Public Houses', fontsize=14, fontweight='bold')
ax.annotate(
    "Note the concentration of pubs along transport routes and in central areas",
    xy=(0.5, 0.02),
    xycoords='figure fraction',
    ha='center',
    fontsize=9,
    bbox=dict(boxstyle="round,pad=0.3", fc="white", ec="gray", alpha=0.8)
)

plt.tight_layout()
plt.show()

## 5. Saving Maps to Files

Once you've created a well-designed map, you'll often need to save it for reports, presentations, or websites. GeoPandas and Matplotlib provide several options for exporting maps.

### Common Export Formats

- **PNG (.png)**: Good for web use, supports transparency
- **JPEG (.jpg)**: Smaller file size but doesn't support transparency
- **PDF (.pdf)**: Vector format, good for print and scalability
- **SVG (.svg)**: Vector format, good for web and further editing

### Considerations for Map Export

- **Resolution**: Set appropriate DPI (dots per inch) based on intended use
- **Dimensions**: Consider the aspect ratio and physical size
- **Metadata**: Include titles, sources, and creation dates
- **Legend**: Ensure the legend is clear and properly positioned

## 6. Choropleth Maps: Visualizing Data Across Areas

**Choropleth maps** use color to represent statistical values across geographic areas, making them one of the most common and effective ways to visualize spatial patterns in data.

### What is a Choropleth Map?

A choropleth map displays divided geographical areas that are colored, shaded, or patterned in relation to a data variable. This allows us to see patterns that might not be obvious in a table of values.

### Key Considerations for Creating Effective Choropleths

- **Normalization**: Raw counts should usually be normalized by area or population
- **Classification Method**: How to group continuous data into color categories
- **Color Scheme**: Choosing appropriate colors for the data type
- **Visual Perception**: Ensuring the map is intuitively readable

Let's use an Index of Multiple Deprivation (IMD) dataset for London to demonstrate various choropleth techniques. The IMD is a UK government measure of relative deprivation across small areas.

In [ ]:
# Load London IMD data
# IMD (Index of Multiple Deprivation) is a UK government measure of relative deprivation
# MSOAs (Middle Super Output Areas) are geographic boundaries with ~7,500 residents
imd_london = gpd.read_file('data/shp/London_IMD_MSOA.shp')
print(f"Number of MSOAs: {len(imd_london)}")
imd_london.head()

In [ ]:
# Examine available columns
print("Available data fields:")
for col in imd_london.columns:
    if col != 'geometry':
        print(f"- {col}")

### 6.1 Categorical Choropleths

Categorical choropleths are used when your data represents distinct categories rather than a continuous scale. They use different colors to distinguish between categories.

For categorical data:
- Use a **qualitative color scheme** where colors are visually distinct
- Ensure categories are clearly defined and mutually exclusive
- Include a clear legend with category names

Let's create a map showing areas with majority white population versus other areas:

In [ ]:
# Create a categorical variable based on demographic data
# The 'white_p' column contains the percentage of white population in each MSOA
imd_london['majority_race'] = np.where(imd_london.eval("white_p > 50"), "White majority", "Other majority")

# Count areas in each category
print(imd_london['majority_race'].value_counts())

In [ ]:
# Create a categorical choropleth with improved design
fig, ax = plt.subplots(1, figsize=(12, 8), subplot_kw=dict(aspect='equal'))

ax.axis('off')

# Define a custom color palette for categorical data
category_colors = {'White majority': '#6baed6', 'Other majority': '#fd8d3c'}

# Plot categorical data with custom colors
imd_london.plot(
    ax=ax,
    column='majority_race',
    categorical=True,
    cmap=CATEGORICAL_CMAP,  # Using consistent categorical colormap
    alpha=0.9, 
    edgecolor='white', 
    linewidth=0.2, 
    legend=True,
    legend_kwds={'title': 'Population Composition'}
)

plt.title('London: Racial Majority by Middle Super Output Area', fontsize=14, fontweight='bold')

# Add explanatory note
ax.annotate(
    "Based on 2011 Census data - MSOAs are areas with ~7,500 residents",
    xy=(0.5, 0.02),
    xycoords='figure fraction',
    ha='center',
    fontsize=9,
    bbox=dict(boxstyle="round,pad=0.3", fc="white", ec="gray", alpha=0.8)
)

plt.tight_layout()
plt.show()

When creating categorical choropleths, note these key points:
- Use the `column` parameter to specify which data to visualize
- Set `categorical=True` to treat the data as categories rather than continuous values
- Setting `legend=True` adds a legend with category names
- Choose a qualitative colormap appropriate for categorical data (e.g., 'Set1', 'Set2', 'Set3', 'Pastel1', etc.)
- Consider color associations and accessibility when selecting colors

The map reveals clear spatial patterns in London's demographic distribution, with outer areas generally having higher white populations compared to inner London.

### 6.2 Choropleth Classification Methods

For continuous data (like income, temperature, or population density), we need to convert the continuous scale into discrete color bins. How we divide the data into these bins can significantly affect the visual interpretation of the map.

There are several classification methods available, each with its own strengths and weaknesses:

1. **Equal Interval**: Divides the range into equal-sized intervals
   - *Advantage*: Easy to understand and interpret
   - *Disadvantage*: May not reflect natural groupings in the data
   - *Best for*: Data that is uniformly distributed

2. **Quantiles**: Creates bins with equal number of observations in each
   - *Advantage*: Works well with skewed data
   - *Disadvantage*: May place similar values in different categories
   - *Best for*: Showing relative rankings in the data

3. **Fisher-Jenks**: Minimizes variance within classes, maximizes variance between classes
   - *Advantage*: Finds "natural breaks" in the data
   - *Disadvantage*: More computationally intensive
   - *Best for*: Data with clear clustering

Let's prepare a dataset attribute to visualize:

In [ ]:
# Select an attribute to visualize from the IMD data
imd_attribute = 'crime'  # Crime score from IMD (higher values indicate higher crime levels)

# Examine the distribution of this variable
fig, ax = plt.subplots(figsize=(10, 6))
sns.histplot(imd_london[imd_attribute], kde=True, ax=ax)
plt.title(f'Distribution of {imd_attribute} scores across London MSOAs', fontsize=14)
plt.xlabel(f'{imd_attribute.capitalize()} Score (higher = more crime)')
plt.ylabel('Frequency')
plt.grid(alpha=0.3)
plt.show()

# Display basic statistics
print(f"\nBasic statistics for {imd_attribute} scores:")
print(imd_london[imd_attribute].describe())

In [ ]:
# Examine how Equal Interval classification bins the data
classes = EqualInterval(imd_london[imd_attribute], k=7)
print(f"Equal Interval Classification with {classes.k} classes")
print("Class counts:")
print(classes.counts)
print("\nClass break points:")
print(classes.bins)

In [ ]:
# View the bin boundaries for easier understanding
print("Equal Interval Classification Bins:")
for i in range(len(classes.bins)-1):
    print(f"Class {i+1}: {classes.bins[i]:.2f} to {classes.bins[i+1]:.2f}")

In [ ]:
# Create a helper function to visualize classification schemes
def draw_bins(geodf, attribute, scheme):
    """Visualize data distribution with classification break points"""
    schemes = {
        'equal_interval': EqualInterval, 
        'quantiles': Quantiles, 
        'fisher_jenks': FisherJenks
    }
    
    classi = schemes[scheme](geodf[attribute], k=7)
    
    # Set up the figure
    fig, ax = plt.subplots(1, figsize=(10, 6))
    
    # Plot the kernel density estimation (KDE)
    sns.kdeplot(geodf[attribute], fill=True, color='#4575b4', alpha=0.7)
    
    # Add a tick for every value at the bottom of the plot (rugs)
    sns.rugplot(geodf[attribute], alpha=0.3, color='#4575b4')
    
    # Loop over each break point and plot a vertical line
    for cut in classi.bins:
        plt.axvline(cut, color='red', linewidth=0.75, alpha=0.8, linestyle='--')
        plt.text(cut, plt.ylim()[1]*0.9, f'{cut:.1f}', 
                 rotation=90, ha='right', fontsize=8)
    
    plt.title(f"{scheme.replace('_', ' ').title()} Classification of {attribute.capitalize()} Scores", fontsize=14)
    plt.xlabel(attribute.capitalize())
    plt.ylabel("Density")
    plt.grid(alpha=0.3)
    
    # Display image
    plt.tight_layout()
    plt.show()

# Visualize Equal Interval classification
draw_bins(imd_london, imd_attribute, 'equal_interval')

The visualization above shows how Equal Interval divides the data range into equal segments. Notice that most observations fall into the lower bins, while few areas have high values. This is typical of skewed distributions.

Let's create a choropleth map using Equal Interval classification:

In [ ]:
# Create an Equal Interval choropleth with improved design
fig, ax = plt.subplots(1, figsize=(12, 8), subplot_kw=dict(aspect='equal'))

ax.axis('off')
ax.set_title("London Crime Index (Equal Interval Classification)", fontsize=14, fontweight='bold')

# Plot choropleth with Equal Interval classification
choropleth = imd_london.plot(
    ax=ax,
    column=imd_attribute, 
    scheme='equal_interval', 
    k=7, 
    cmap=SEQUENTIAL_CMAP,  # Using consistent sequential colormap 
    alpha=0.9, 
    edgecolor='white', 
    linewidth=0.2, 
    legend=True,
    legend_kwds={'title': 'Crime Score', 'loc': 'lower right', 'frameon': True}
)

# Improve legend appearance
legend = ax.get_legend()
legend.set_bbox_to_anchor((0.98, 0.25))
frame = legend.get_frame()
frame.set_facecolor('white')
frame.set_alpha(0.8)

# Add data source information
ax.text(
    0.99, 0.01,
    'Classification Method: Equal Interval\n'
    'Higher scores indicate higher crime levels\n'
    'Data: UK Index of Multiple Deprivation',
    ha='right', va='bottom',
    size=8,
    color='#555555',
    transform=ax.transAxes,
    bbox=dict(boxstyle="round,pad=0.3", fc="white", ec="gray", alpha=0.8)
)

plt.tight_layout()
plt.show()

### 6.3 Quantiles Classification

Quantiles classification creates bins with an equal number of observations in each, which can better highlight patterns when data is skewed. This approach ensures that each color on your map represents roughly the same number of geographic units.

Key features of Quantiles classification:
- Each class contains approximately the same number of features
- Works well for comparing relative rankings
- May place similar values in different classes if data is clustered

Let's examine how Quantiles would classify our crime data:

In [ ]:
# Examine Quantiles classification
classes = Quantiles(imd_london[imd_attribute], k=7)
print(f"Quantiles Classification with {classes.k} classes")
print("Class counts:")
print(classes.counts)

Let's examine the bin boundaries for Quantiles classification:

In [ ]:
# View the bin boundaries for Quantiles
print("Quantiles Classification Bins:")
for i in range(len(classes.bins)-1):
    print(f"Class {i+1}: {classes.bins[i]:.2f} to {classes.bins[i+1]:.2f}")

The visualization of the distribution with Quantiles classification shows how this method creates bins with equal numbers of observations:

In [ ]:
# Visualize Quantiles classification
draw_bins(imd_london, imd_attribute, 'quantiles')

Notice how Quantiles creates break points that ensure an equal number of areas in each category. This results in narrower bins where there are many observations (in the lower range of values) and wider bins where there are fewer observations (in the higher range).

Now let's create a choropleth map using Quantiles classification:

In [ ]:
# Create a helper function for choropleth maps
def draw_choropleth(geodf, attribute, scheme, cmap=SEQUENTIAL_CMAP, title=None, figsize=(12, 8)):
    """Create a choropleth map with the specified classification scheme"""
    fig, ax = plt.subplots(1, figsize=figsize, subplot_kw=dict(aspect='equal'))

    ax.axis('off')
    
    if title:
        ax.set_title(title, fontsize=14, fontweight='bold')

    # Plot choropleth
    geodf.plot(
        ax=ax,
        column=attribute, 
        scheme=scheme, 
        k=7, 
        cmap=cmap,  # Now using the parameter with SEQUENTIAL_CMAP as default
        alpha=0.9, 
        edgecolor='white', 
        linewidth=0.2, 
        legend=True,
        legend_kwds={'title': attribute.capitalize() + ' Score', 'loc': 'lower right', 'frameon': True}
    )

    # Improve legend appearance
    legend = ax.get_legend()
    legend.set_bbox_to_anchor((0.98, 0.25))
    frame = legend.get_frame()
    frame.set_facecolor('white')
    frame.set_alpha(0.8)

    # Add data source information
    ax.text(
        0.99, 0.01,
        f'Classification Method: {scheme.replace("_", " ").title()}\n'
        f'Higher scores indicate higher {attribute} levels\n'
        f'Data: UK Index of Multiple Deprivation',
        ha='right', va='bottom',
        size=8,
        color='#555555',
        transform=ax.transAxes,
        bbox=dict(boxstyle="round,pad=0.3", fc="white", ec="gray", alpha=0.8)
    )

    plt.tight_layout()
    plt.show()

# Create a Quantiles choropleth
draw_choropleth(
    imd_london, 
    imd_attribute, 
    'quantiles', 
    title="London Crime Index (Quantiles Classification)"
)

### 6.4 Fisher-Jenks Classification

Fisher-Jenks (also known as "natural breaks") is a more sophisticated method that tries to find natural break points in the data. It minimizes the variance within classes and maximizes the variance between classes.

Key features of Fisher-Jenks classification:
- Identifies "natural" groupings in the data
- Groups similar values together
- Maximizes the differences between classes
- Often produces the most intuitive maps for human interpretation

Let's examine how Fisher-Jenks would classify our crime data:

In [ ]:
# Examine Fisher-Jenks classification
classes = FisherJenks(imd_london[imd_attribute], k=7)
print(f"Fisher-Jenks Classification with {classes.k} classes")
print("Class counts:")
print(classes.counts)

In [ ]:
# View the bin boundaries for Fisher-Jenks
print("Fisher-Jenks Classification Bins:")
for i in range(len(classes.bins)-1):
    print(f"Class {i+1}: {classes.bins[i]:.2f} to {classes.bins[i+1]:.2f}")

In [ ]:
# Visualize Fisher-Jenks classification
draw_bins(imd_london, imd_attribute, 'fisher_jenks')

Fisher-Jenks finds natural clusters in the data. Notice how the break points are placed at locations where there are natural gaps in the data distribution.

Now let's create a choropleth map using Fisher-Jenks classification:

In [ ]:
# Create a Fisher-Jenks choropleth
draw_choropleth(
    imd_london, 
    imd_attribute, 
    'fisher_jenks', 
    title="London Crime Index (Fisher-Jenks Classification)"
)

In [ ]:
# Create a function to compare different classification schemes side by side
def plot_scheme(scheme, var, db, figsize=(16, 6), saveto=None):
    '''
    Plot the distribution over value and geographical space of variable `var` using scheme `scheme`
    
    Arguments
    ---------
    scheme   : str
               Name of the classification scheme to use 
    var      : str
               Variable name 
    db       : GeoDataFrame
               Table with input data
    figsize  : Tuple
               [Optional. Default = (16, 6)] Size of the figure to be created.
    saveto   : None/str
               [Optional. Default = None] Path for file to save the plot.
    '''
    schemes = {'equal_interval': EqualInterval, 
               'quantiles': Quantiles, 
               'fisher_jenks': FisherJenks}
    classi = schemes[scheme](db[var], k=7)
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=figsize, dpi=100)
    
    # KDE plot
    sns.kdeplot(db[var], fill=True, color='#26828E', ax=ax1)
    sns.rugplot(db[var], alpha=0.5, color='#26828E', ax=ax1)
    for cut in classi.bins:
        ax1.axvline(cut, color='#31688E', linewidth=0.75)
    ax1.set_title('Value distribution')
    ax1.grid(alpha=0.3)
    
    # Map
    db.plot(column=var, scheme=scheme, alpha=0.75, k=7, 
           cmap=SEQUENTIAL_CMAP, ax=ax2, linewidth=0.1,  # Using consistent colormap
           legend=True, legend_kwds={'title': var.capitalize()})
    ax2.axis('equal')
    ax2.set_axis_off()
    ax2.set_title('Geographical distribution')
    
    f.suptitle(scheme.replace('_', ' ').title(), size=20)
    f.tight_layout()
    
    if saveto:
        plt.savefig(saveto)
    plt.show()

# Use the function to plot the Equal Interval scheme
plot_scheme('equal_interval', imd_attribute, imd_london)

In [ ]:
# Create a function to compare classification schemes side-by-side
def plot_scheme_comparison(var, db, figsize=(16, 8)):
    """
    Plot different classification schemes side by side
    """
    schemes = ['equal_interval', 'quantiles', 'fisher_jenks']
    
    # Create a figure with 3 subplots
    fig, axes = plt.subplots(1, 3, figsize=figsize)
    
    # For each classification scheme
    for i, scheme in enumerate(schemes):
        ax = axes[i]
        
        # Plot the choropleth
        db.plot(
            column=var, 
            scheme=scheme, 
            k=7, 
            cmap=SEQUENTIAL_CMAP,  # Using consistent sequential colormap
            alpha=0.9, 
            edgecolor='white', 
            linewidth=0.2,
            ax=ax,
            legend=True,
            legend_kwds={'title': var.capitalize() if i == len(schemes)-1 else None, 'loc': 'lower right'}
        )
        
        # Format the subplot
        ax.set_title(f"{scheme.replace('_', ' ').title()}", fontsize=12)
        ax.axis('off')
    
    # Add an overall title
    fig.suptitle(f"Comparison of Classification Methods: {var.replace('_', ' ').title()} Score", 
                 fontsize=16, y=0.95)
    
    plt.tight_layout()
    
    # Add annotation explaining the differences
    plt.figtext(
        0.5, 0.01,
        "Equal Interval: Equal value ranges | Quantiles: Equal number of observations | Fisher-Jenks: Natural breaks in data",
        ha="center", fontsize=10, bbox=dict(boxstyle="round,pad=0.3", fc="white", ec="gray", alpha=0.8)
    )
    
    plt.show()

# Compare all three classification schemes
plot_scheme_comparison(imd_attribute, imd_london)

In [ ]:
# Focusing on Specific Areas with Inset Maps

# Select specific boroughs to focus on
boroughs_selected = ['E09000007', 'E09000033','E09000019','E09000020','E09000001']
scheme = 'quantiles'

# Load London boroughs
boroughs_df = gpd.read_file("data/shp/London_Borough_Excluding_MHW.shp")
print(f"Total number of boroughs: {len(boroughs_df)}")
print(f"Selected boroughs: {len(boroughs_selected)}")

# Filter to selected boroughs
boroughs_df = boroughs_df.loc[boroughs_df['GSS_CODE'].isin(boroughs_selected)]
print(f"Selected borough names: {boroughs_df['NAME'].tolist()}")

# Create the main figure
fig, ax = plt.subplots(1, figsize=(12, 9), subplot_kw=dict(aspect='equal'))

# Plot selected areas with choropleth
selected_data = imd_london.loc[imd_london['lad11cd'].isin(boroughs_selected)]
selected_data.plot(
    ax=ax,
    alpha=0.9,
    column=imd_attribute,
    k=7,
    cmap=SEQUENTIAL_CMAP,  # Using consistent sequential colormap
    linewidth=0.1,
    edgecolor='black',
    legend=True, 
    scheme=scheme,
    legend_kwds={'title': 'Crime Score'}
)

# Add borough boundaries for context
boroughs_df.plot(ax=ax, linewidth=1.5, color="none", edgecolor="#333333")

# Add borough labels
for idx, row in boroughs_df.iterrows():
    # Get centroid of the borough
    centroid = row['geometry'].centroid
    # Add the borough name
    ax.annotate(
        row['NAME'],
        xy=(centroid.x, centroid.y),
        ha='center',
        fontsize=9,
        fontweight='bold',
        color='black',
        bbox=dict(boxstyle="round,pad=0.3", fc="white", ec="gray", alpha=0.8)
    )

ax.axis('off')
ax.set_title(f"Crime Index in Selected London Boroughs", fontsize=14, fontweight='bold')

# Add information text
ax.text(
    0.99, 0.01,
    f'Classification Method: {scheme.title()}\nHigher scores indicate higher crime levels\nData: UK Index of Multiple Deprivation',
    ha='right', va='bottom',
    size=8,
    color='#555555',
    transform=ax.transAxes,
    bbox=dict(boxstyle="round,pad=0.3", fc="white", ec="gray", alpha=0.8)
)

# Create an inset axes for the overview map
axins = inset_axes(
    ax,
    width="25%",  # width = % of parent_bbox
    height=2.0,
    loc=2,  # Upper left corner
)

# Plot the entire London area in the inset
imd_london.plot(
    ax=axins,
    color='#cccccc',
    edgecolor='#999999',
    linewidth=0.2,
)

# Highlight the selected area in the inset
union = boroughs_df.geometry.union_all()
union_gdf = gpd.GeoDataFrame(geometry=[union])
union_gdf.plot(ax=axins, linewidth=1, edgecolor='red', color='red', alpha=0.5)

# Add title to the inset map
axins.set_title("Location in London", fontsize=8)
axins.axis('off')

plt.tight_layout()
plt.show()

## 7. Advanced Techniques: Map Zooming and Focus Areas

When working with complex geographic datasets, it's often useful to focus on specific regions of interest while maintaining context. There are multiple approaches for this.

In [ ]:
# Simple zooming by setting axis limits
fig, ax = plt.subplots(1, figsize=(8, 6), subplot_kw=dict(aspect='equal'))

ax.axis('off')

# Plot the data
imd_london.plot(
    ax=ax,
    column=imd_attribute, 
    scheme='quantiles', 
    k=7, 
    cmap=SEQUENTIAL_CMAP,  # Using consistent sequential colormap
    edgecolor='black',
    linewidth=0.05,
    alpha=0.9,
    legend=True,
    legend_kwds={'title': 'Crime Score'}
)

plt.title("Zoomed View of Central London Crime Rates", fontsize=14, fontweight='bold')

# Add context annotation
ax.text(
    0.5, 0.02,
    "Zoomed view of central area showing finer detail",
    ha='center',
    transform=ax.transAxes,
    fontsize=9,
    bbox=dict(boxstyle="round,pad=0.3", fc="white", ec="gray", alpha=0.8)
)

plt.tight_layout()
plt.show()

In [ ]:
# Create a dual map with main area and context
fig, axs = plt.subplots(1, 2, figsize=(14, 6), gridspec_kw={'width_ratios': [2, 1]})

# Left panel - Zoomed area
ax_main = axs[0]
ax_main.axis('off')
ax_main.set_title("Central London Crime Rates (Detail)", fontsize=12, fontweight='bold')

# Plot data in main panel
imd_london.plot(
    ax=ax_main,
    column=imd_attribute, 
    scheme='quantiles', 
    k=7, 
    cmap=SEQUENTIAL_CMAP,  # Using consistent sequential colormap
    edgecolor='black',
    linewidth=0.05,
    alpha=0.9,
    legend=True,
    legend_kwds={'title': 'Crime Score'}
)

# Right panel - Context map
ax_context = axs[1]
ax_context.axis('off')
ax_context.set_title("London Context Map", fontsize=12, fontweight='bold')

# Plot full map for context
imd_london.plot(
    ax=ax_context,
    column=imd_attribute, 
    scheme='quantiles', 
    k=7, 
    cmap=SEQUENTIAL_CMAP,  # Using consistent sequential colormap
    edgecolor='black',
    linewidth=0.05,
    alpha=0.9
)

# Highlight the zoomed area on the context map
rect = plt.Rectangle(
    (530000, 175000), 
    10000, 10000, 
    fill=False, 
    edgecolor='red', 
    linewidth=2
)
ax_context.add_patch(rect)

# Add arrow pointing to the zoomed area
ax_context.annotate(
    'Zoomed Area',
    xy=(535000, 180000),
    xytext=(510000, 190000),
    arrowprops=dict(facecolor='red', shrink=0.05, width=2)
)

plt.tight_layout()
plt.show()

## 8. Interactive Mapping with Folium

Static maps are useful for reports and publications, but interactive maps provide a more engaging experience for web-based presentations. The Folium library combines the power of Python with the interactive mapping capabilities of Leaflet.js.

Key advantages of interactive maps:
- Zoom and pan capabilities
- Popup information on click
- Layer toggling
- Base map options
- Ability to export as HTML for web sharing

Let's create an interactive choropleth map of our crime data:

In [ ]:
# Import Folium for interactive mapping
import folium
from folium.features import GeoJsonPopup, GeoJsonTooltip
import branca.colormap as cm
import json

# We need to convert to WGS84 (EPSG:4326) coordinate system for web mapping
imd_london_wgs84 = imd_london.to_crs(epsg=4326)

# Create a center point for the map
center = [imd_london_wgs84.geometry.centroid.y.mean(), 
          imd_london_wgs84.geometry.centroid.x.mean()]

# Create a custom color scale
colormap = cm.linear.YlOrRd_09.scale(
    imd_london[imd_attribute].min(),
    imd_london[imd_attribute].max()
)

# Create a tooltip to show on hover
tooltip = GeoJsonTooltip(
    fields=['msoa11nm', 'lad11nm', imd_attribute],
    aliases=['MSOA Name:', 'Borough:', 'Crime Score:'],
    localize=True,
    sticky=False,
    labels=True
)

# Create a popup to show on click
popup = GeoJsonPopup(
    fields=['msoa11nm', 'lad11nm', imd_attribute, 'imd'],
    aliases=['MSOA Name:', 'Borough:', 'Crime Score:', 'IMD Rank:'],
    localize=True,
    labels=True
)

# Create the map
m = folium.Map(
    location=center,
    zoom_start=10,
    tiles='CartoDB positron',
    control_scale=True
)

# Add the choropleth layer
folium.Choropleth(
    geo_data=imd_london_wgs84,
    name='Crime Score',
    data=imd_london_wgs84,
    columns=['msoa11nm', imd_attribute],
    key_on='feature.properties.msoa11nm',
    fill_color='YlOrRd',  # Keeping YlOrRd for folium to match our SEQUENTIAL_CMAP
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Crime Score',
    smooth_factor=0.5
).add_to(m)

# Add the GeoJson with tooltips and popups
folium.GeoJson(
    imd_london_wgs84,
    name='Areas',
    style_function=lambda x: {
        'fillColor': 'transparent',
        'color': 'black',
        'weight': 0.5
    },
    tooltip=tooltip,
    popup=popup
).add_to(m)

# Add layer control
folium.LayerControl().add_to(m)

# Add a title
title_html = '''
    <h3 align="center" style="font-family:Arial; margin-top:10px">
        <b>London Crime Score by MSOA</b>
    </h3>
    <p align="center" style="font-family:Arial; font-size:12px">
        Higher values indicate higher crime levels. Source: UK Index of Multiple Deprivation
    </p>
'''
m.get_root().html.add_child(folium.Element(title_html))

# Display the map
display(m)

## 9. Best Practices for Choropleth Maps

Creating effective choropleth maps requires careful consideration of data, design, and communication principles. Here are some best practices to follow:

### Data Preparation

1. **Use normalized data**: Raw counts often reflect population distribution rather than the phenomenon of interest. Normalize by area, population, or another relevant denominator.

2. **Choose appropriate units**: Select geographic units that are appropriate for your data and audience. Too detailed, and patterns may be lost; too coarse, and important variations may be obscured.

3. **Handle missing data carefully**: Missing data should be clearly distinguished from low values. Consider using a separate color or pattern.

### Visual Design

1. **Select appropriate classification method**: Match the method to your data distribution and communication goals:
   - Equal interval for uniformly distributed data
   - Quantiles for highlighting relative ranks
   - Natural breaks for finding inherent groupings

2. **Choose effective color schemes**:
   - Sequential schemes (e.g., light to dark) for quantities
   - Diverging schemes (e.g., red to blue) for data with a meaningful midpoint
   - Qualitative schemes for categorical data
   
3. **Consider color accessibility**: Ensure your maps can be interpreted by people with color vision deficiencies.

4. **Include context**: Add relevant geographic features to orient the reader.

### Communication

1. **Create clear legends**: Use intuitive labels and ensure the legend is easily visible.

2. **Add informative titles and annotations**: Help readers understand what they're seeing.

3. **Include source information**: Document data sources and processing methods.

4. **Consider complementary visualizations**: Pair choropleths with charts showing the underlying data distribution.

By following these best practices, you can create choropleth maps that effectively communicate spatial patterns and relationships in your data.

## 10. Case Study: Analyzing Multiple Factors

Let's create a more complex analysis combining multiple factors from our IMD dataset to identify areas with particular characteristics. This demonstrates how choropleth techniques can be applied to more sophisticated analyses.

We'll create an index that combines crime, health, and education factors to identify areas with particular challenges across multiple dimensions.

In [ ]:
# Create a multi-factor index
# In the IMD data, higher scores indicate more deprivation
imd_london['combined_index'] = (imd_london['crime'] + 
                               imd_london['health'] + 
                               imd_london['education']) / 3

# Create a standard score (z-score) for comparison
imd_london['combined_z'] = (imd_london['combined_index'] - imd_london['combined_index'].mean()) / imd_london['combined_index'].std()

# Create categories based on standard deviations
conditions = [
    (imd_london['combined_z'] < -1.5),
    (imd_london['combined_z'] >= -1.5) & (imd_london['combined_z'] < -0.5),
    (imd_london['combined_z'] >= -0.5) & (imd_london['combined_z'] < 0.5),
    (imd_london['combined_z'] >= 0.5) & (imd_london['combined_z'] < 1.5),
    (imd_london['combined_z'] >= 1.5)
]

categories = [
    'Significantly Below Average',
    'Below Average',
    'Average',
    'Above Average',
    'Significantly Above Average'
]

# Add a default value with the same data type as categories
imd_london['combined_category'] = np.select(conditions, categories, default='Uncategorized')

# Count observations in each category
print(imd_london['combined_category'].value_counts())

# Create a categorical choropleth
fig, ax = plt.subplots(figsize=(12, 8), subplot_kw=dict(aspect='equal'))
ax.axis('off')

# Define colors for categories (using ColorBrewer scheme)
color_dict = {
    'Significantly Below Average': '#1a9641',
    'Below Average': '#a6d96a',
    'Average': '#ffffbf',
    'Above Average': '#fdae61',
    'Significantly Above Average': '#d7191c'
}

# Create custom categorical legend handles
from matplotlib.patches import Patch
legend_elements = [
    Patch(facecolor=color_dict[cat], edgecolor='black', alpha=0.7, label=cat)
    for cat in categories
]

# Plot the map
imd_london.plot(
    column='combined_category',
    categorical=True,
    cmap='RdYlGn_r',
    ax=ax,
    legend=False,
    alpha=0.7,
    edgecolor='white',
    linewidth=0.2
)

# Add custom legend
ax.legend(handles=legend_elements, title='Combined Deprivation Level',
          loc='lower right', frameon=True, framealpha=0.9)

ax.set_title('London: Combined Crime, Health, and Education Challenges', 
             fontsize=14, fontweight='bold')

# Add explanatory note
ax.text(
    0.5, 0.02,
    "Higher values (red) indicate greater challenges across crime, health, and education dimensions.",
    ha='center',
    transform=ax.transAxes,
    fontsize=9,
    bbox=dict(boxstyle="round,pad=0.3", fc="white", ec="gray", alpha=0.8)
)

plt.tight_layout()
plt.show()


## 11. Summary and Next Steps

In this tutorial, we've covered:

1. **Basic Spatial Data Handling**:
   - Loading different types of spatial data (points, lines, polygons)
   - Understanding and exploring spatial datasets
   - Converting between coordinate reference systems

2. **Map Creation and Styling**:
   - Creating basic maps
   - Controlling aesthetics (colors, lines, transparency)
   - Multi-layer mapping techniques

3. **Choropleth Maps**:
   - Different classification methods (Equal Interval, Quantiles, Fisher-Jenks)
   - Visualizing categorical vs. continuous data
   - Comparing classification schemes
   - Best practices for choropleth design

4. **Advanced Techniques**:
   - Zooming into specific regions
   - Creating inset maps for context
   - Interactive web mapping
   - Multi-factor analysis

### Next Steps for Further Learning

1. **Explore spatial analysis**: Go beyond visualization to spatial statistics, clustering, and modeling

2. **Learn interactive mapping**: Dive deeper into libraries like Folium, Plotly, or Bokeh for web-based maps

3. **Try different data types**: Work with raster data, geocoding, and network analysis

4. **Integrate with other tools**: Combine GeoPandas with machine learning libraries or web frameworks

5. **Create reproducible workflows**: Build automated pipelines for spatial data processing and visualization

By mastering these techniques, you'll be well-equipped to create informative, visually compelling maps that effectively communicate spatial patterns and relationships in your data.